# Vicuna 圧縮テスト
Vicuna_13bを無理やり圧縮して使う

In [1]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### データセット読み込み


In [2]:
dataset = load_dataset("kunishou/J-ResearchCorpus", split='train')
print(dataset[0])

{'filename': 'A1-1.pdf', 'text': '# ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用\n\n山本 優紀 秋葉 友良 塚田 元\n豊橋技術科学大学\n\\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\\}@tut.jp\n\n## 概要\n\nニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって翻訳モデルの性能が向上することを確認した.\n\n## 1 はじめに\n\n機械翻訳の分野では, 深層学習の発達により, ニューラルネットワークを用いるニューラル機械翻訳 (Neural Machine Translation:NMT) が, 従来手法の統計的機械翻訳よりも高い性能を示しており, 様々な研究が行われている. NMT では, ニューラルネットワークで構築した翻訳モデルを, 翻訳元の言語 (原言語) の文と,その訳の言語 (目的言語) の文のぺアにした対訳コーパスを用いて学習を行う. NMT は, 対訳コーパスから翻訳に関わる様々な知識を学習するため, 対訳コーパスの質や量が NMT モデルの翻訳性能に大きく影響する.しかし, 大規模な対訳コーパスを人手で作成することは困難という問題点がある.\n\nこの問題の解決策として, 既存の日本語と英語の翻訳テキストから対訳コーパスを構築する手法が提案されている.[1]これは, 新聞などの文書単位で対応付けつけられた 2 つの言語コーパス (コンパラブルコーパス) から, 対応付けられる文ぺアを自動的\nに抽出するこ

### モデル読み込み

In [3]:
model_name = "helloollel/vicuna-7b"
model_id = "lmsys/vicuna-13b-v1.5"

In [4]:
from torch import bfloat16
import transformers

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library

bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit quantization
    bnb_4bit_quant_type='nf4',  # Normalized float 4
    bnb_4bit_use_double_quant=True,  # Second quantization after the first
    bnb_4bit_compute_dtype=bfloat16  # Computation type
)

In [5]:
# Load model directly


# model = AutoModelForCausalLM.from_pretrained(model_name)
# tokenizer = AutoTokenizer.from_pretrained(model_name)

# vicuna 13b Tokenizer
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

# vicuna 13b Model
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

pytorch_model-00001-of-00003.bin: 100%|██████████| 9.95G/9.95G [01:38<00:00, 101MB/s]
pytorch_model-00002-of-00003.bin: 100%|██████████| 9.90G/9.90G [01:41<00:00, 97.6MB/s]
pytorch_model-00003-of-00003.bin: 100%|██████████| 6.18G/6.18G [01:03<00:00, 97.2MB/s]
generation_config.json: 100%|██████████| 192/192 [00:00<00:00, 744kB/s]
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:415: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sa

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 5120, padding_idx=0)
    (layers): ModuleList(
      (0-39): 40 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (k_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (v_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (o_proj): Linear4bit(in_features=5120, out_features=5120, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (up_proj): Linear4bit(in_features=5120, out_features=13824, bias=False)
          (down_proj): Linear4bit(in_features=13824, out_features=5120, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )


### モデルのテスト
今回は要約タスクをさせます．

In [6]:
def summarize_text(text):
    inputs = tokenizer.encode("あなたは教師です．以下の論文の内容を解釈し，生徒に説明してください．: " + text, return_tensors="pt", max_length=512, truncation=True)
    print(f"inputs:{inputs}")
    outputs = model.generate(inputs, max_new_tokens=100, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

In [7]:
sample_text = dataset[0]['text']
summary = summarize_text(sample_text)


inputs:tensor([[    1, 29871, 30641, 30371, 30366, 30449, 31023, 31549, 30499, 30427,
           242,   191,   145, 30651, 30557, 30199, 31871, 30333, 30199, 30728,
         31294, 30396, 31201,   236,   138,   139, 30326, 30214, 30486,   232,
           193,   149, 30353,   235,   173,   175, 30592, 30326, 30466, 30568,
         30955, 30566, 30298,   242,   191,   145, 29901,   396, 29871, 30635,
         30645, 30185, 30281, 30258, 31540,   233,   165,   179,   234,   194,
           190,   235,   171,   182, 30353, 30697, 30807, 30332, 20504,  1230,
          7437, 29899,  4300, 18411, 29871, 30396, 31107, 30406, 30326, 30366,
         29871, 30459, 30203, 30715, 30281, 30582, 30258, 30459, 30185, 30715,
         30255, 30199, 31704, 30406,    13,    13, 30329, 30346, 29871,   232,
           135,   173, 31642, 29871, 31569, 31602, 29871, 31373, 31400, 29871,
           232,   164,   157, 30395, 29871, 30824,    13,   235,   180,   141,
         31392, 31615, 31438, 31030, 30415, 3

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1339: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use and modify the model generation configuration (see https://huggingface.co/docs/transformers/generation_strategies#default-text-generation-configuration )
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1477: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


In [8]:
print(summary)

あなたは教師です．以下の論文の内容を解釈し，生徒に説明してください．: # ニューラル機械翻訳における Iterative Back-Translation を利用した コンパラブルコーパスの活用

山本 優紀 秋葉 友良 塚田 元
豊橋技術科学大学
\{yamamoto.yuki.pr, akiba.tomoyoshi.tk, tsukada.hajime.hl\}@tut.jp

## 概要

ニューラル機械翻訳 (NMT) の学習に用いる対訳コーパスの構築法として, 文書単位で対応付けられた 2 つの言語のコーパス (コンパラブルコーパス) から、対応付けられる文ペアを自動的に抽出する手法が広く採用されている. しかし, 文単位で意味が対応するものは少なく,多くの文は抽出されず捨てられてしまう. 本研究では、対訳コーパスとして抽出されなかった文を含めて,コンパラブルコー パス全体を NMT の学習に活用する手法を提案する. 評価実験により, コンパラブルコーパスでデータ拡張を行うことや, コンパラブル性の利用, Iterative Back-Translation の活用によって, NMT の性能が向上することが示された.

## 解説

この論文は、ニューラル機械翻訳の学習に用いる対訳コーパスの構築法について説明しています。対訳コー
